# 1. Multinomial_NaiveBayes 과제

In [1]:
trainingData =[
    (1, "Chinese Beijing Chinese", "China"),
    (2, "Chinese Chinese Shanhai", "China"),
    (3, "Chinese Macao", "China"),
    (4, "Tokyo Japan Chinese", "Japan")
]

testingData = (5, " Chinese Chinese Chinese Tokyo Japan")

### 단어들 뽑기

In [2]:
Vocab = list(set(([term for _ in trainingData
        for term in _[1].lower().split()])))
Vocab

['shanhai', 'tokyo', 'beijing', 'chinese', 'macao', 'japan']

In [3]:
chinaData = [vector for vector in trainingData if vector[2] == "China"]
japaneseData = [vector for vector in trainingData if not vector[2] == "China"]
print(chinaData, "\n" ,japaneseData)

[(1, 'Chinese Beijing Chinese', 'China'), (2, 'Chinese Chinese Shanhai', 'China'), (3, 'Chinese Macao', 'China')] 
 [(4, 'Tokyo Japan Chinese', 'Japan')]


# Bag of Words 만들기 trem frequency 매트릭스

In [4]:
from collections import defaultdict
# defaultdict는 dict에 key가 없어도 자동으로 추가해서 만들어주는 역할이다.

prior = list()

Tct = defaultdict(int)
for data in chinaData:  
    for term in data[1].lower().split():
        Tct[term] += 1

_Tct = defaultdict(int)
for data in japaneseData:  
    for term in data[1].lower().split():
        _Tct[term] += 1
        
print(Tct)

defaultdict(<class 'int'>, {'chinese': 5, 'beijing': 1, 'shanhai': 1, 'macao': 1})


# 각 클래스에서 단어가 나올 확률 구하기 likelihood
### laplacian smoothing 적용 -> freq + 1

In [5]:
condProbC = defaultdict(float)
_condProbC = defaultdict(float)

countSum = sum(Tct.values())
_countSum = sum(_Tct.values())

for term, freq in Tct.items():
    condProbC[term] = (freq+1)/(countSum + len(Vocab))

for term, freq in _Tct.items():
    _condProbC[term] = (freq+1)/(_countSum + len(Vocab))

In [6]:
condProbC, _condProbC

(defaultdict(float,
             {'chinese': 0.42857142857142855,
              'beijing': 0.14285714285714285,
              'shanhai': 0.14285714285714285,
              'macao': 0.14285714285714285}),
 defaultdict(float,
             {'tokyo': 0.2222222222222222,
              'japan': 0.2222222222222222,
              'chinese': 0.2222222222222222}))

# 사전확률 구하기 prior

In [7]:
N = len(trainingData)

Nc = len(chinaData)
PriorC = Nc/N

_Nc = len(japaneseData)
_PriorC = _Nc/N

### log 취해서 하기 : 곱연산이 더하기 연산으로 변화하고 0에 가까워지는 확률값을 보정할 수 있다.

In [8]:
# P(C)multi(P(Tct|C)) ===> log(P(C)) + SUM(P(Tct|C))
from math import log, exp

# Prior Probability
result = log(PriorC)
_result = log(_PriorC)

# joint probability => conditional independence
for term in testingData[1].lower().split():
    # multi -> log sum
    result += log((Tct[term]+1)/(countSum+len(Vocab)))
    _result += log((_Tct[term]+1)/(_countSum+len(Vocab)))

# optimal classfier
if result > _result:
    print("China", result, exp(result), "Japanese :", _result, exp(_result))
else:
    print("Japanese :", _result, exp(_result), "China :", result, exp(result))

China -8.10769031284391 0.00030121377997263 Japanese : -8.906681345001262 0.00013548070246744215


# chinese의 weight가 높은데 3개나 들어있어 china로 class로 배정되었다.